In [79]:
from openbb import obb
import pandas as pd
import numpy as np
from scipy.stats import norm
import datetime as dt

In [89]:
def FetchRates(Start_Date=None, End_Date=None):
    treasury_data = obb.fixedincome.government.treasury_rates(start_date=Start_Date, end_date=End_Date, provider="federal_reserve").to_df()
    fed_funds = obb.fixedincome.rate.effr(start_date=Start_Date, end_date=End_Date, provider="federal_reserve").to_df()[['rate']].rename(columns={'rate': 'FedFunds'})
    sofr_data = obb.fixedincome.rate.sofr(start_date=Start_Date, end_date=End_Date, provider="federal_reserve").to_df()[['rate']].rename(columns={'rate': 'SOFR'})

    #Merge
    rates_data = treasury_data.join([fed_funds, sofr_data], how='outer')
    #Reset Index
    rates_data = rates_data.rename_axis('Date').reset_index()

    #Renaming Columns
    rates_data.rename(columns={
        "month_1": "1Mo", "month_2": "2Mo", "month_3": "3Mo", "month_6": "6Mo", 
        "year_1": "1Yr", "year_2": "2Yr", "year_3": "3Yr", 
        "year_5": "5Yr", "year_7": "7Yr", "year_10": "10Yr", "year_30": "30Yr"
    }, inplace=True)

    # Filter and Sort
    cols = ['Date', 'FedFunds', 'SOFR', '1Mo', '3Mo', '6Mo', '1Yr', '2Yr', '3Yr', '5Yr', '7Yr', '10Yr', '30Yr']
    rates_data = rates_data[cols].sort_values('Date')

    # Removing rows where primary benchmarks are missing (weekends/holidays)
    rates_data = rates_data.dropna(subset=['FedFunds', '1Mo'])
    
    # Forward Fill SOFR (Modern syntax)
    rates_data['SOFR'] = rates_data['SOFR'].ffill()
    
    return rates_data

In [12]:
# Filter for rows that contain any NaN values
#null_rows = rates_data[rates_data.isnull().any(axis=1)]
#null_rows

In [91]:
start_date = "2020-01-02"
end_date = dt.datetime.today().strftime('%Y-%m-%d') 
FetchRates(start_date, end_date)

,Date,FedFunds,SOFR,1Mo,3Mo,6Mo,1Yr,2Yr,3Yr,5Yr,7Yr,10Yr,30Yr
0,2020-01-02,0.0155,0.0154,0.0153,0.0154,0.0157,0.0156,0.0158,0.0159,0.0167,0.0179,0.0188,0.0233
1,2020-01-03,0.0155,0.0155,0.0152,0.0152,0.0155,0.0155,0.0153,0.0154,0.0159,0.0171,0.0180,0.0226
2,2020-01-06,0.0155,0.0155,0.0154,0.0156,0.0156,0.0154,0.0154,0.0156,0.0161,0.0172,0.0181,0.0228
3,2020-01-07,0.0155,0.0156,0.0152,0.0154,0.0156,0.0153,0.0154,0.0155,0.0162,0.0174,0.0183,0.0231
4,2020-01-08,0.0155,0.0155,0.0150,0.0154,0.0156,0.0155,0.0158,0.0161,0.0167,0.0178,0.0187,0.0235
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,2025-12-16,0.0364,0.0369,0.0376,0.0364,0.0360,0.0351,0.0348,0.0352,0.0369,0.0390,0.0415,0.0482
1490,2025-12-17,0.0364,0.0369,0.0373,0.0364,0.0359,0.0351,0.0349,0.0353,0.0370,0.0391,0.0416,0.0483
1491,2025-12-18,0.0364,0.0366,0.0371,0.0362,0.0358,0.0349,0.0346,0.0350,0.0366,0.0387,0.0412,0.0480
1492,2025-12-19,0.0364,0.0366,0.0371,0.0362,0.0360,0.0351,0.0348,0.0353,0.0370,0.0391,0.0416,0.0482
